# Cell/particle Counting and scoring stained objects

This notebook demonstrates how to process using
[CellProfiler](http://cellprofiler.org/) plates
associated to the paper
*Integration of biological data by kernels on graph nodes allows prediction
of new genes involved in mitotic chromosome condensation.*
See http://dx.doi.org/10.1091/mbc.E13-04-0221.
We use the example pipeline
*Cell/particle counting, and scoring the percentage of stained objects*
see http://cellprofiler.org/examples/#PercentPositive.
This pipeline is for two channels images.

### Import Packages

In [ ]:
# Import Cell Profiler Dependencies
import cellprofiler.preferences as cpprefs
import cellprofiler.pipeline as cpp

# Inject Image module used to inject OMERO image planes into
# Cell Profiler Pipeline
from cellprofiler.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
from omero.gateway import BlitzGateway
from omero.grid import DoubleColumn, ImageColumn, LongColumn, WellColumn
from omero.constants.namespaces import NSBULKANNOTATIONS
from omero.gateway import FileAnnotationWrapper
from omero.model import OriginalFileI

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

# Import Matplotlib
import matplotlib

from getpass import getpass
from os.path import expanduser
cpprefs.set_headless()


### Set Cell Output Directory

In [ ]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials

In [ ]:
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST)
conn.connect()

### Fetch OMERO Plate that contains the Images to be analysed

In [ ]:
# To be modified
plate_id = 102
plate = conn.getObject("Plate", plate_id)
print('Plate Name: ', plate.getName())

In [ ]:
# Load pipeline and inspect modules
home = expanduser("~")
pipeline = cpp.Pipeline()
file = "/notebooks/CellProfiler/pipelines/ExamplePercentPositive.cppipe"
pipeline.load(home+file)

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)

print('Pipeline modules:')
for module in pipeline.modules():
    print(module.module_num, module.module_name)

### Run Cell Profiler Pipeline on a plate

In [ ]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
files = list()

# create list from generator
wells = list(plate.listChildren())
# use the first 5 wells only
wells = wells[0:5]
well_count = len(wells)

for count, well in enumerate(wells):
    print('Well: %s/%s' % (count + 1, well_count),
          'row:', well.row, 'column:', well.column)
    # Load a single Image per Well
    image = well.getImage(0)
    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()
    # Inject image for each Channel (pipeline only handles 2 channels)
    for c in range(0, size_c):

        plane = pixels.getPlane(0, c, 0)
        image_name = image.getName()

        # Name of the channel expected in the pipeline
        if c == 0:
            image_name = 'OrigBlue'
        if c == 1:
            image_name = 'OrigGreen'

        inject_image_module = InjectImage(image_name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    m = pipeline_copy.run()

    # Results obtained as CSV from Cell Profiler
    path = new_output_directory + '/Nuclei.csv'
    f = pandas.read_csv(path, index_col=None, header=0)
    f['Image'] = image.getId()
    f['Well'] = well.getId()
    f['Cell_Count'] = len(f.index)
    files.append(f)

Nuclei = pandas.concat(files, ignore_index=True)

### Calculate statistics

In [ ]:
Nuclei.describe()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (32.0, 30.0)
# Drop few columns
df = Nuclei.drop(['Image', 'ImageNumber', 'Well', 'ObjectNumber',
                  'Number_Object_Number', 'Classify_PH3Neg',
                  'Classify_PH3Pos'], axis=1)
df.hist()

### Save the ouput to OMERO as OMERO.table

In [ ]:
cols = []
summary = Nuclei.groupby('Image').mean()
# Free memory
del Nuclei
for col in summary.columns:
    if col == 'Image':
        cols.append(ImageColumn(col, '', summary[col]))
    elif col == 'Well':
        cols.append(WellColumn(col, '', summary[col]))
    elif summary[col].dtype == 'int64':
        cols.append(LongColumn(col, '', summary[col]))
    elif summary[col].dtype == 'float64':
        cols.append(DoubleColumn(col, '', summary[col]))

resources = conn.c.sf.sharedResources()
repository_id = resources.repositories().descriptions[0].getId().getValue()
table_name = "idr0002_cellprofiler"
table = resources.newTable(repository_id, table_name)
table.initialize(cols)
table.addData(cols)

# Link the table to the plate
orig_file = table.getOriginalFile()
file_ann = FileAnnotationWrapper(conn)
file_ann.setNs(NSBULKANNOTATIONS)
file_ann._obj.file = OriginalFileI(orig_file.id.val, False)
file_ann.save()
plate.linkAnnotation(file_ann)
table.close()

### Close the connection to the OMERO server

In [ ]:
conn.close()

### License
Copyright (C) 2019 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.